In [ ]:
%%capture
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [ ]:
import pandas as pd
import numpy as np
import requests
from pyproj import Transformer

df = pd.read_csv('fired_conus_ak_2000_to_2024_events.csv')

transformer = Transformer.from_crs("epsg:32610", "epsg:4326", always_xy=True)  # Adjust EPSG if UTM zone varies
df['lon'], df['lat'] = transformer.transform(df['ig_utm_x'].values, df['ig_utm_y'].values)

df['ig_date'] = pd.to_datetime(df['ig_date'])
df['last_date'] = pd.to_datetime(df['last_date'])

def get_event_dates(row):
    return [row['ig_date'] + pd.Timedelta(days=i) for i in range(int(row['event_dur']))]

df['event_dates'] = df.apply(get_event_dates, axis=1)

print(df.head(1))

# weather_df = df.apply(enrich_row, axis=1)

# df = pd.concat([df, weather_df], axis=1)
# df.head()

In [ ]:
import json
import requests
import numpy as np
import pandas as pd
import time

def get_weather(lat, lon, start_date, end_date):
    time.sleep(1)
    # url = "https://archive-api.open-meteo.com/v1/archive"
    url = "https://customer-archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": [
            "temperature_2m_mean",              # Avg temperature
            "precipitation_sum",                # Total precipitation
            "wind_speed_10m_max",               # Max wind speed
            "wind_gusts_10m_max",               # Max wind gusts
            "wind_direction_10m_dominant",      # Dominant wind direction
            "sunshine_duration",                # Sunshine duration
            "pressure_msl_max",                 # Max sea level pressure (optional)
            "pressure_msl_min",                  # Min sea level pressure (optional),
            "relative_humidity_2m_mean",         # Avg relative humidity
        ],
        "timezone": "auto",
        "apikey": "OvVpRVbn10xCC8K5"
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        daily = data['daily']

        # Use the first day of returned data
        idx = 0
        augmented_data = {
            'tavg': daily['temperature_2m_mean'][idx],
            'prcp': daily['precipitation_sum'][idx],
            'wspd': daily['wind_speed_10m_max'][idx],
            'wdir': daily['wind_direction_10m_dominant'][idx],
            'wpgt': daily['wind_gusts_10m_max'][idx],
            'pres_max': daily.get('pressure_msl_max', [None])[idx],
            'pres_min': daily.get('pressure_msl_min', [None])[idx],
            'tsun': daily.get('sunshine_duration', [None])[idx],
            'relative_humidity_2m_mean': daily.get('relative_humidity_2m_mean', [None])[idx]
        }

        return augmented_data
    else:
        print(f"Error fetching weather data: HTTP {response.status_code}, {response.text}")
        # print(json.dumps(response.json(), indent=2))
        return None

def enrich_row(row):
    start = row['ig_date'].strftime('%Y-%m-%d')
    end = row['last_date'].strftime('%Y-%m-%d')
    weather_data = get_weather(row['lat'], row['lon'], start, end)
    if weather_data:
        print(f"Getting weather info for {start}")
        # print(f"Weather data: {json.dumps(weather_data, indent=2)}")
        return pd.Series(weather_data)
    else:
        return pd.Series({key: None for key in [
            'tavg', 'prcp', 'wspd', 'wdir', 'wpgt', 'pres_max', 'pres_min', 'tsun', 'relative_humidity_2m_mean'
        ]})


In [ ]:
# Apply the enrichment function to each row
weather_df = df.apply(enrich_row, axis=1)

# Concatenate weather data to the original DataFrame
df = pd.concat([df, weather_df], axis=1)
df.head()

In [ ]:
df.to_csv('fired_conus_with_weather.csv', index=False)